In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [ ]:
!pip install mlflow

In [ ]:
import mlflow
import mlflow.pytorch

In [ ]:
# Verifies the existance of GPU instance
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
from torchvision import models

**Initializing pretrained Models**

In [ ]:
inception = models.inception_v3(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

In [ ]:
for param in inception.parameters():
    param.requires_grad = False

In [ ]:
print(inception)

In [ ]:
# To modify the final output layer/architecture and also the auxilliary section of the network 
aux_in_features = inception.AuxLogits.fc.in_features   
inception.AuxLogits.fc = nn.Linear(aux_in_features, 2)

In [ ]:
for param in inception.parameters():
    if param.requires_grad:
        print(param.shape)

In [ ]:
in_features = inception.fc.in_features
inception.fc =  nn.Linear(in_features, 2)

In [ ]:
in_features = inception.fc.in_features
inception.fc =  nn.Sequential(
    nn.Linear(in_features,1000),
    nn.ReLU(),
    nn.Linear(1000,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,2)
              )

----------------

In [ ]:
resnet = models.resnet18(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
print(resnet)

In [ ]:
for param in resnet.parameters():
    param.requires_grad = False

In [ ]:
in_features = resnet.fc.in_features
resnet.fc = nn.Sequential(
    nn.Linear(in_features,1000),
    nn.ReLU(),
    nn.Linear(1000,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,2)
              )

In [ ]:
for param in resnet.parameters():
    if param.requires_grad:
        print(param.shape)

-----------------------

In [ ]:
vgg = models.vgg16_bn(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
print(vgg)

In [ ]:
for param in vgg.parameters():
    param.requires_grad = False

In [ ]:
final_in_features = vgg.classifier[6].in_features
vgg.classifier[6] = nn.Sequential(
    nn.Linear(final_in_features,1000),
    nn.ReLU(),
    nn.Linear(1000,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,2)
              )

**Data Importation**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torchvision import datasets, transforms

In [ ]:
import copy

In [ ]:
import numpy as np

- The datsets class from PyTorch expects a folder such that each class of i/p data has its own sub-folder
- Splitting data into : Train/Validation/Test ~ (60-20-20) %

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(299), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

transform_val = transforms.Compose([
    transforms.RandomResizedCrop(299), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(299), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

In [ ]:
train = datasets.ImageFolder(root= '/content/drive/MyDrive/Data_ThesisProject/Train',transform=transform_train)
test = datasets.ImageFolder(root= '/content/drive/MyDrive/Data_ThesisProject/Test',transform=transform_test)
val = datasets.ImageFolder(root= '/content/drive/MyDrive/Data_ThesisProject/Validation',transform=transform_val)

In [ ]:
batch_size = 16
trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=True)

- The dataset loaders would by default provide the output encodings depending on thier classes, since the inputs were given such that a separate folder exists for each class 
- Here the first folder is given 0 class and correspondingly the second folder is given 1 class i.e 
 - Defective class : 0
 - Non-Defective class : 1

**Model Importation**
- To evaluate the performnce on multiple models

In [ ]:
model = inception.to(device)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
for data in trainloader:
  ip,labels = data
  ip,labels = ip.to(device),labels.to(device)
  print(labels)
  break

tensor([1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1], device='cuda:0')


In [ ]:
def evaluation(dataset,model):
  model.eval()
  total = 0
  positive = 0
  for data in dataset:
    inputs,labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    pred_op,aux_op = model(inputs)
    pred = torch.argmax(pred_op,1)
    total += labels.size(0)
    positive += (pred==labels).sum().item()
  return 100 *(positive/total)

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
def fit(trainloader,testloader,model,loss_fn,optim,epochs=35):
  min_loss = 1000
  loss_arr = []
  for i in tqdm_notebook(range(epochs),total=epochs,unit='epoch'):
    ct = 0
    for data in trainloader:
      ct+=1
      inputs,labels = data
      inputs, labels = inputs.to(device), labels.to(device)
      pred_op,aux_op = model(inputs)
      loss = loss_fn(pred_op,labels)+0.3*(loss_fn(aux_op,labels))
      opt.zero_grad()
      loss.backward()
      opt.step()

      if min_loss>loss.item():
        min_loss = loss.item()
        best_model = copy.deepcopy(inception.state_dict())
        print(min_loss)
      
      if ct%100 == 0:
        print(ct,loss.item())
      
      del inputs,labels,pred_op
      torch.cuda.empty_cache()

    loss_arr.append(loss.item())

    print(i,'Test_acc:',evaluation(testloader,inception),'Train_acc:',evaluation(trainloader,inception))
    plt.plot(loss_arr)
    plt.show()

In [ ]:
fit(trainloader,testloader,inception,loss_fn,optim,epochs=35)

------------------------

In [ ]:
vgg = vgg.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(vgg.parameters(), lr=0.05)

In [ ]:
def evaluation(dataset,model):
  model.eval()
  total = 0
  positive = 0
  for data in dataset:
    inputs,labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    pred_op= model(inputs)
    pred = torch.argmax(pred_op,1)
    total += labels.size(0)
    positive += (pred==labels).sum().item()
  return 100 *(positive/total)

In [ ]:
def fit(trainloader,testloader,model,loss_fn,optim,epochs=35):
  min_loss = 1000
  loss_arr = []
  for i in tqdm_notebook(range(epochs),total=epochs,unit='epoch'):
    ct = 0
    for data in trainloader:
      ct+=1
      inputs,labels = data
      inputs, labels = inputs.to(device), labels.to(device)
      pred_op = model(inputs)

      loss = loss_fn(pred_op,labels)
      opt.zero_grad()
      loss.backward()
      opt.step()

      if min_loss>loss.item():
        min_loss = loss.item()
        best_model = copy.deepcopy(inception.state_dict())
        print(min_loss)
      
      if ct%100 == 0:
        print(ct,loss.item())
      
      del inputs,labels,pred_op
      torch.cuda.empty_cache()

    loss_arr.append(loss.item())

    print(i,'Test_acc:',evaluation(testloader,model),'Train_acc:',evaluation(trainloader,model))
    plt.plot(loss_arr)
    plt.show()

In [ ]:
fit(trainloader,testloader,vgg,loss_fn,opt,epochs=35)

-------------------

In [ ]:
resnet = resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(resnet.parameters(), lr=0.05)

In [ ]:
def evaluation(dataset,model):
  model.eval()
  total = 0
  positive = 0
  for data in dataset:
    inputs,labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    pred_op= model(inputs)
    pred = torch.argmax(pred_op,1)
    total += labels.size(0)
    positive += (pred==labels).sum().item()
  return 100 *(positive/total)

In [ ]:
def fit(trainloader,testloader,model,loss_fn,optim,epochs=15):
  min_loss = 1000
  loss_arr = []
  for i in tqdm_notebook(range(epochs),total=epochs,unit='epoch'):
    ct = 0
    for data in trainloader:
      ct+=1
      inputs,labels = data
      inputs, labels = inputs.to(device), labels.to(device)
      pred_op = model(inputs)

      loss = loss_fn(pred_op,labels)
      opt.zero_grad()
      loss.backward()
      opt.step()

      if min_loss>loss.item():
        min_loss = loss.item()
        best_model = copy.deepcopy(inception.state_dict())
        print(min_loss)
      
      if ct%100 == 0:
        print(ct,loss.item())
      
      del inputs,labels,pred_op
      torch.cuda.empty_cache()

    loss_arr.append(loss.item())

    print(i,'Test_acc:',evaluation(testloader,model),'Train_acc:',evaluation(trainloader,model))
    plt.plot(loss_arr)
    plt.show()

In [ ]:
fit(trainloader,testloader,resnet,loss_fn,opt,epochs=15)

-----------------------------------------
**Hyper Parameter Tuning**

In [ ]:
class Parameters():
  def __init__(self,epochs,log_interval):
    self.epochs = epochs
    self.log_interval = log_interval
args = Parameters(15,16)

In [ ]:
batch_size = 16
trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=True)

**Train Function**

In [ ]:
def train(epoch):
  model.train()
  for i,data in enumerate(trainloader):
    ip,labels = data
    ip,labels = ip.to(device),labels.to(device)
    op,aux_op = model(ip)
    loss = loss_fn(op,labels)+0.3*(loss_fn(aux_op,labels))
    opt.zero_grad()
    loss.backward()
    opt.step()

    if i%args.log_interval == 0:
      #mlflow.log_metric('train_loss',loss.data.item())
      print('Train epoch : {} [{}/{}]\tLoss: {:.3f}'.format(epoch,(i*len(ip)),len(trainloader.dataset),loss.data.item()))

**Test Function**

In [ ]:
def test(epoch):
  #model.eval()
  correct = 0

  with torch.no_grad():

    for data in valloader:
      ip1,labels1 = data
      ip1,labels1 = ip1.to(device),labels1.to(device)
      op1,aux_op1 = model(ip1)
      test_loss = loss_fn(op1,labels1)+0.3*(loss_fn(aux_op1,labels1))
      pred_op = torch.argmax(op1,1)
      correct += (pred_op==labels1).sum().item()
    test_acc = correct/len(valloader.dataset)
    print('Accuracy (Test):{}%'.format(test_acc))
    #mlflow.log_metric('test_loss',test_loss)
    #mlflow.log_metric('test_accuracy',test_acc)

**Loop for Tuning of Parameters**

In [ ]:
for lr in [0.01,0.02,0.05]:
  for momentum in [0.9,0.95,0.99]:
    epochs = args.epochs
    loss_fn = nn.CrossEntropyLoss()
    opt = optim.SGD(model.parameters(), lr=lr, momentum = momentum)

    #with mlflow.start_run() as run:
      #for i,j in vars(args).items():
        #mlflow.log_param(i,j)
      #mlflow.log_param('lr',lr)
      #mlflow.log_param('momentum',momentum)

    for i in tqdm_notebook(range(epochs),total=epochs,unit='epoch'):
      train(i)
      test(i)
    print('lr:',lr,'momentum:',momentum)

In [ ]:
vars(args).items()

dict_items([('epochs', 5), ('log_interval', 20)])